In [1]:
import requests
import json
import time
import math
import os
from random import choice
import re

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [ ]:
BASE_URL = 'https://www.fairprice.com.sg/'

In [ ]:
def random_headers():
    '''
    Choose a random user_agent to use as an header for requests
    list of http header fields: https://en.wikipedia.org/wiki/List_of_HTTP_header_fields
    '''
    user_agents = ['Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.99 Safari/537.36',
                   'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.99 Safari/537.36',
                   'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.99 Safari/537.36',
                   'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_1) AppleWebKit/602.2.14 (KHTML, like Gecko) Version/10.0.1 Safari/602.2.14',
                   'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.71 Safari/537.36',
                   'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.98 Safari/537.36',
                   'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.98 Safari/537.36',
                   'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.71 Safari/537.36',
                   'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.99 Safari/537.36',
                   'Mozilla/5.0 (Windows NT 10.0; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0',
                   'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.181 Safari/537.36']

    # return {'User-Agent': choice(user_agents), 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8'}
    return {'User-Agent': choice(user_agents), 'Accept': 'text/html'}


In [ ]:
def requestForPage(url):
    # headers = {'User-Agent': ''}
    r = requests.get(url, headers=random_headers())
    return r.text


# Getting the Links

In [ ]:
soup = BeautifulSoup(requestForPage(BASE_URL), 'lxml')

In [ ]:
soup.find('a', {'class':'selSecondNav'}).get('href')

In [ ]:
top_links = [link.get('href') for link in soup.find_all('a', {'class':'selSecondNav'})]
top_links

# Selenium

In [2]:
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--window-size=1920x1080")
chrome_driver = os.path.join(
    os.getcwd(),'..', '..', 'WebScraping', 'seleniumdrivers', 'chromedriver')
browser = webdriver.Chrome(
        chrome_options=chrome_options, executable_path=chrome_driver)

In [3]:
browser.get('https://www.fairprice.com.sg/beer-wine-spirits/beer/ale-lager-malt')

In [ ]:
soup = BeautifulSoup(browser.page_source, 'lxml')
soup.find('div', id= 'noFilterMsg')['class'][0]

In [ ]:
browser.get('https://www.fairprice.com.sg/beer-wine-spirits/beer/alcopop-cider')

In [ ]:
soup = BeautifulSoup(browser.page_source, 'lxml')
soup.find('div', id='noFilterMsg')['class'][0]

In [ ]:
'prodLoadMoreBck hide_element' in browser.page_source

In [ ]:
'prodLoadMoreBck' in browser.page_source

In [ ]:
while True:
    if 'prodLoadMoreBck hide_element'in browser.page_source:
        break
    elif 'prodLoadMoreBck' in browser.page_source:
        try:
            browser.find_element_by_link_text('LOAD MORE').click()
        except:
            pass

# Getting data

In [ ]:
def clean_text(text):
    return re.sub('\s+', '', text)

In [ ]:
browser.get('https://www.fairprice.com.sg/product/prego-pasta-sauce-tomato-basil-garlic-300g-10876057')

In [ ]:
soup = BeautifulSoup(browser.page_source, 'lxml')

In [ ]:
image_url = 'https://s3-ap-southeast-1.amazonaws.com/www8.fairprice.com.sg/fpol/media/images/product/XL/'

In [ ]:
soup.find('div', class_ = 'pdpImgWrapper').img['data-large-img-url'].split('/')[-1]

In [ ]:
soup.find('h1').text

In [ ]:
clean_text(soup.find('span', class_ = 'pdt_weightMg').text)

In [ ]:
clean_text(soup.find('span', class_ = 'pdt_C_price' ).text)

In [ ]:
print(re.sub('\s+', '',soup.find('div', class_ = 'pdt_desc_d_row pdpDesc clearfix').div.text))
print(soup.find('div', class_ = 'pdt_desc_d_row pdpDesc clearfix').p.text.split('•')[1:])

In [ ]:
for i in soup.find_all('div', class_ = 'pdt_desc_d_row clearfix'):
    print(clean_text(i.find('div', class_ = 'pdr_name').text))
    print(clean_text(i.find('div', class_ = 'pdr_p_desc').text))

# Another product

In [ ]:
browser.get('https://www.fairprice.com.sg/product/rio-cocktail-rose-whisky-275ml-13109817')

In [ ]:
soup = BeautifulSoup(browser.page_source, 'lxml')

In [ ]:
image_url = 'https://s3-ap-southeast-1.amazonaws.com/www8.fairprice.com.sg/fpol/media/images/product/XL/'

In [ ]:
image_url + soup.find('div', class_ = 'pdpImgWrapper').img['data-large-img-url'].split('/')[-1]

In [ ]:
from IPython.display import Image
Image(image_url + soup.find('div', class_ = 'pdpImgWrapper').img['data-large-img-url'].split('/')[-1])

In [ ]:
soup.find('h1').text

In [ ]:
clean_text(soup.find('span', class_ = 'pdt_weightMg').text)

In [ ]:
clean_text(soup.find('span', class_ = 'pdt_C_price' ).text)

In [ ]:
print(re.sub('\s+', '',soup.find('div', class_ = 'pdt_desc_d_row pdpDesc clearfix').div.text))
print(soup.find('div', class_ = 'pdt_desc_d_row pdpDesc clearfix').p.text.split('•')[1:])

In [ ]:
for i in soup.find_all('div', class_ = 'pdt_desc_d_row clearfix'):
    print(clean_text(i.find('div', class_ = 'pdr_name').text))
    print(clean_text(i.find('div', class_ = 'pdr_p_desc').text))